# Charts for my website

- Author: Kiril from Mindgraph
- Last meaningful update: 15-04-2025

This notebook creates some `plotly` charts for [my website](www.mindgraph.dk). The charts are displayed as visuals in this notebook and also exported to `.html` files on GitHub so that they can be directly embedded on the website.

In [35]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio

In [36]:
# Importing data
sales_data = pd.read_parquet("Output data/FinalSalesAndIncome.parquet")
indexed_development = pd.read_parquet("Output data/IndexedDevelopment.parquet")

# Getting min and max years for historical data
min_hist = sales_data[sales_data["PriceType"] == "Actual price"]["Year"].min()
max_hist = sales_data[sales_data["PriceType"] == "Actual price"]["Year"].max()

In [37]:
# # Changing to the default color palette in plotly
# pio.templates.default = "plotly"

In [38]:
# Creating a custom color palette with the MG colors
# Using the "Classy" palette from: https://mycolor.space/?hex=%231EA2B5&sub=1
my_custom_palette = ["#1ea2b5", "#324b4f", "#95b0b5", "#9f8ac3", "#6b588d"]
my_template = pio.templates["plotly_white"].layout.template
my_template.layout.colorway = my_custom_palette
pio.templates["my_custom_template"] = my_template
pio.templates.default = "my_custom_template"

## Template functions for chart

If we need to recreate the same chart with different subsamples of the data, it makes more sense to store the chart as a function:

In [39]:
def plot_price(
    sales_data: pd.DataFrame,
    metric_for_use: str,
    chart_lang: str,
    selected_locations: list[str],
    price_types: list[str],
    chart_title: str | None = None,
    center_legend: bool = True,
    increase_y_margin: bool = True,
) -> None:

    # Preparing data for chart
    data_to_display = sales_data.copy()
    data_to_display = data_to_display[
        data_to_display["Municipality"].isin(selected_locations)
    ]
    data_to_display = data_to_display[
        data_to_display["PriceType"].isin(price_types)
    ].copy()
    data_to_display["Kommune"] = data_to_display["Municipality"]

    # Rounding off to 2 decimals
    data_to_display[metric_for_use] = np.round(data_to_display[metric_for_use], 2)

    # Sorting and cleaning up
    data_to_display.sort_values("Year", ascending=False, inplace=True)
    data_to_display.reset_index(inplace=True, drop=True)

    # Identifying the selected municipalities
    unique_mncp = data_to_display["Municipality"].unique()

    # Checking whether multiple municipalities are selected
    if len(unique_mncp) > 1:
        if chart_lang == "en":
            color_var_for_chart = "Municipality"
        else:
            color_var_for_chart = "Kommune"
    else:
        color_var_for_chart = None

    # Preparing text-based descriptions of the metrics
    if metric_for_use == "AvgSalesPrice":
        if chart_lang == "en":
            metric_for_use_txt = "Average price per m²"
        else:
            metric_for_use_txt = "Gennemsnitlig pris pr. m²"
    elif metric_for_use == "AvgSalesPriceIdx":
        if chart_lang == "en":
            metric_for_use_txt = "Average price per m² (indexed)"
        else:
            metric_for_use_txt = "Gennemsnitlig pris pr. m² (indekseret)"
    elif metric_for_use == "M2AffordedTotal":
        if chart_lang == "en":
            metric_for_use_txt = "m² buyable with annual disposable income"
        else:
            metric_for_use_txt = "m² som kan købes med en årlig disponibel indkomst"
    elif metric_for_use == "YearsoBuy50M2Total":
        if chart_lang == "en":
            metric_for_use_txt = "Years of annual income needed to buy 50 m²"
        else:
            metric_for_use_txt = "Antal indkomstår, som kræves for at købe 50 m²"
    else:
        if chart_lang == "en":
            metric_for_use_txt = "Unknown"
        else:
            metric_for_use_txt = "Ukendt"

    # Plotting the data on a chart
    year_label = "Year" if chart_lang == "en" else "År"
    fig = px.line(
        data_to_display,
        x="Year",
        y=metric_for_use,
        color=color_var_for_chart,
        labels={
            "Year": year_label,
            metric_for_use: metric_for_use_txt,
        },
    )
    if chart_title:
        fig.update_layout({"title": chart_title})
    fig.update_traces(textposition="top center")

    # Optional centering the legend on top of the chart
    if center_legend:
        fig.update_layout(
            legend=dict(
                x=0.5,  # Position along the x-axis (0.5 is the center)
                y=1.1,  # Position above the plot area (greater than 1 to move it above)
                xanchor="center",  # Center the legend horizontally
                yanchor="bottom",  # Align the bottom of the legend to the y=1.1 position
                bgcolor="rgba(255, 255, 255, 0.5)",  # Transparent background for the legend
            ),
        )

    fig
    return fig

## Figure 1

In [40]:
# Setting user parameters
n_mncp_to_show = 15
price_types = ["Actual price"]
# chart_title = f"The {n_mncp_to_show} municipalities with the highest price increase between {min_hist}-{max_hist}"

In [41]:
# Filtering the data and sorting in the appropriate order
data_for_plot = sales_data.copy()
data_for_plot = data_for_plot[data_for_plot["Year"] == max_hist].copy()
data_for_plot = data_for_plot[
    data_for_plot["Municipality"] != "National average"
].copy()
data_for_plot.sort_values("AvgSalesPriceChange", ascending=False, inplace=True)
data_for_plot.reset_index(inplace=True, drop=True)
data_for_plot = data_for_plot[:n_mncp_to_show].copy()
data_for_plot.sort_values("AvgSalesPriceChange", inplace=True)

### English

In [42]:
# Plotting the data on a chart
fig = px.bar(
    data_for_plot,
    x="AvgSalesPriceChange",
    y="Municipality",
    orientation="h",
    # title=f"Top {n_mncp_to_show} municipalities in {max_hist} with the highest change in price per m² relative to 1992",
    labels={
        "Municipality": "Municipality",
        "AvgSalesPriceChange": "% change in average price per m²",
    },
)

# Adjust the Y-axis font size to fit the labels
fig.update_layout(
    margin=dict(l=120),  # Increase margin to accommodate longer text labels
    yaxis=dict(
        tickfont=dict(size=10),  # Reduce the font size of the Y-axis labels
    ),
)

fig

# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure1_EN.html", "w", encoding="utf-8") as f:
    f.write(html_str)

# # Exporting as a PNG file (requires the "kaleido" package)
# fig.write_image("Charts/Figure1_EN.png")

# Previewing
fig

### Danish

In [43]:
# Plotting the data on a chart
fig = px.bar(
    data_for_plot,
    x="AvgSalesPriceChange",
    y="Municipality",
    orientation="h",
    labels={
        "Municipality": "Kommune",
        "AvgSalesPriceChange": "% ændring i gennemsnitlig pris pr. m²",
    },
)

# Adjust the Y-axis font size to fit the labels
fig.update_layout(
    margin=dict(l=120),  # Increase margin to accommodate longer text labels
    yaxis=dict(
        tickfont=dict(size=10),  # Reduce the font size of the Y-axis labels
    ),
)

fig

# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure1_DK.html", "w", encoding="utf-8") as f:
    f.write(html_str)

# Exporting as a PNG file (requires the "kaleido" package)
# fig.write_image("Charts/Figure1_DK.png")

# Previewing
fig

## Figure 2

In [44]:
# Setting user parameters
n_mncp_to_show = 15
price_types = ["Actual price"]
# chart_title = f"The {n_mncp_to_show} most expensive municipalities for buying a flat in {max_hist}"

In [45]:
# Filtering the data and sorting in the appropriate order
data_for_plot = sales_data[sales_data["Year"] == max_hist].copy()
data_for_plot = data_for_plot[
    data_for_plot["Municipality"] != "National average"
].copy()
data_for_plot.sort_values("AvgSalesPrice", ascending=False, inplace=True)
data_for_plot.reset_index(inplace=True, drop=True)
data_for_plot = data_for_plot[:n_mncp_to_show].copy()
data_for_plot.sort_values("AvgSalesPrice", inplace=True)

### English

In [46]:
# Plotting the data on a chart
fig = px.bar(
    data_for_plot,
    x="AvgSalesPrice",
    y="Municipality",
    orientation="h",
    # title=f"Top {n_mncp_to_show} most expensive municipalities in {max_hist}",
    labels={
        "Municipality": "Municipality",
        "AvgSalesPrice": "Average sales price per m²",
    },
)

# Adjust the Y-axis font size to fit the labels
fig.update_layout(
    margin=dict(l=120),  # Increase margin to accommodate longer text labels
    yaxis=dict(
        tickfont=dict(size=10),  # Reduce the font size of the Y-axis labels
    ),
)

# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure2_EN.html", "w", encoding="utf-8") as f:
    f.write(html_str)

# Exporting as a PNG file
# fig.write_image("Charts/Figure2_EN.png")

# Previewing
fig

### Danish

In [47]:
# Plotting the data on a chart
fig = px.bar(
    data_for_plot,
    x="AvgSalesPrice",
    y="Municipality",
    orientation="h",
    # title=f"Top {n_mncp_to_show} dyreste kommuner i {max_hist}",
    labels={
        "Municipality": "Kommune",
        "AvgSalesPrice": "Gennemsnitlig salgspris pr. m²",
    },
)

# Adjust the Y-axis font size to fit the labels
fig.update_layout(
    margin=dict(l=120),  # Increase margin to accommodate longer text labels
    yaxis=dict(
        tickfont=dict(size=10),  # Reduce the font size of the Y-axis labels
    ),
)

# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure2_DK.html", "w", encoding="utf-8") as f:
    f.write(html_str)

# Exporting as a PNG file
# fig.write_image("Charts/Figure2_DK.png")

# Previewing
fig

## Figure 3

In [48]:
# Setting user parameters
price_types = ["Actual price"]
# chart_title_en = (
#     "Indexed sales price, national disposable income and national GDP over time"
# )
# chart_title_dk = (
#     "Indekseret salgspris, national disponibel indkomst og national BNP over tid"
# )

In [49]:
# Preparing data for chart
data_to_display = indexed_development[
    indexed_development["Municipality"] == "National average"
]
data_to_display = data_to_display[data_to_display["PriceType"].isin(price_types)].copy()

# Sorting and cleaning up
data_to_display.sort_values("Year", ascending=False, inplace=True)
data_to_display.reset_index(inplace=True, drop=True)

# Rounding all relevant metrics off to 2 decimals
for metric in [
    "AvgSalesPriceIdx",
    "AvgDispIncomeIdx",
    "GDPIdx",
]:
    data_to_display[metric] = np.round(data_to_display[metric], 2)

# Converting the data to the long format before plotting
value_vars = ["AvgSalesPriceIdx", "AvgDispIncomeIdx", "GDPIdx"]
vars_mapping = {
    "AvgSalesPriceIdx": "Average sales price (indexed)",
    "AvgDispIncomeIdx": "Average disposable income (indexed)",
    "GDPIdx": "GDP (indexed)",
}
data_to_display = pd.melt(
    data_to_display,
    id_vars="Year",
    value_vars=value_vars,
    var_name="Indicator",
    value_name="Index",
)
data_to_display["Indicator"] = data_to_display["Indicator"].map(vars_mapping)

### English

In [50]:
# Plotting the data on a chart
fig = px.line(
    data_to_display,
    x="Year",
    y="Index",
    color="Indicator",
)
# fig.update_layout({"title": chart_title_en})
fig.update_traces(textposition="top center")

# Customize the layout to position the legend in the middle on top
fig.update_layout(
    legend=dict(
        x=0.5,  # Position along the x-axis (0.5 is the center)
        y=1.1,  # Position above the plot area (greater than 1 to move it above)
        xanchor="center",  # Center the legend horizontally
        yanchor="bottom",  # Align the bottom of the legend to the y=1.1 position
        bgcolor="rgba(255, 255, 255, 0.5)",  # Transparent background for the legend
    ),
)

# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure3_EN.html", "w", encoding="utf-8") as f:
    f.write(html_str)

# Exporting as a PNG file
# fig.write_image("Charts/Figure3_EN.png")

# Previewing
fig

### Danish

In [51]:
# Renaming labels for chart
translations = {
    "Average sales price (indexed)": "Gns. salgspris (indekseret)",
    "Average disposable income (indexed)": "Gns. disponibel indkomst (indekseret)",
    "GDP (indexed)": "BNP (indekseret)",
    "Year": "År",
    "Index": "Indeks",
    "Indicator": "Indikator",
}
data_to_display["Indicator"] = data_to_display["Indicator"].map(translations)
data_to_display = data_to_display.rename(columns=translations)

# Plotting the data on a chart
fig = px.line(
    data_to_display,
    x="År",
    y="Indeks",
    color="Indikator",
)
# fig.update_layout({"title": chart_title_dk})
fig.update_traces(textposition="top center")

# Customize the layout to position the legend in the middle on top
fig.update_layout(
    legend=dict(
        x=0.5,  # Position along the x-axis (0.5 is the center)
        y=1.1,  # Position above the plot area (greater than 1 to move it above)
        xanchor="center",  # Center the legend horizontally
        yanchor="bottom",  # Align the bottom of the legend to the y=1.1 position
        bgcolor="rgba(255, 255, 255, 0.5)",  # Transparent background for the legend
    ),
)

# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure3_DK.html", "w", encoding="utf-8") as f:
    f.write(html_str)

# Exporting as a PNG file
# fig.write_image("Charts/Figure3_DK.png")

# Previewing
fig

## Figure 4

In [52]:
# Creating the chart
selected_locations = ["National average"]
# chart_title_en = (
#     "Number of m² that can be bought with annual disposable income (national average)"
# )
# chart_title_dk = (
#     "Antal m² som kan købes med en årlig disponibel indkomst (national gennemsnit)"
# )

### English

In [53]:
fig = plot_price(
    sales_data,
    "M2AffordedTotal",
    "en",
    selected_locations,
    ["Actual price"],
)

# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure4_EN.html", "w", encoding="utf-8") as f:
    f.write(html_str)

# Exporting as a PNG file
# fig.write_image("Charts/Figure4_EN.png")

# Previewing
fig

### Danish

In [54]:
fig = plot_price(
    sales_data,
    "M2AffordedTotal",
    "dk",
    selected_locations,
    ["Actual price"],
)

# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure4_DK.html", "w", encoding="utf-8") as f:
    f.write(html_str)

# Exporting as a PNG file
# fig.write_image("Charts/Figure4_DK.png")

# Previewing
fig

## Figure 5

In [55]:
# Creating the chart
selected_locations = ["København", "Frederiksberg", "Aarhus", "Odense", "Aalborg"]
# chart_title_en = "Number of m² that can be bought with average annual disposable income in Denmark's four largest cities"
# chart_title_dk = "Antal m² som kan købes med en gns. årlig disponibel indkomst i Danmarks fire største byer"

### English

In [56]:
fig = plot_price(
    sales_data,
    "M2AffordedTotal",
    "en",
    selected_locations,
    ["Actual price"],
)

# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure5_EN.html", "w", encoding="utf-8") as f:
    f.write(html_str)

# Exporting as a PNG file
# fig.write_image("Charts/Figure5_EN.png")

# Previewing
fig

### Danish

In [57]:
fig = plot_price(
    sales_data,
    "M2AffordedTotal",
    "dk",
    selected_locations,
    ["Actual price"],
)

# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure5_DK.html", "w", encoding="utf-8") as f:
    f.write(html_str)

# Exporting as a PNG file
# fig.write_image("Charts/Figure5_DK.png")

# Previewing
fig

## Figure 6

In [58]:
# Creating the chart
selected_locations = ["Randers", "Brønderslev", "Faxe"]
# chart_title_en = "Number of m² that can be bought with average annual disposable income in municipalities that do not follow the general trend"
# chart_title_dk = "Antal m² som kan købes med en gns. årlig disponibel indkomst i kommuner, som adskiller sig fra den overordnede tendens"

### English

In [59]:
fig = plot_price(
    sales_data,
    "M2AffordedTotal",
    "en",
    selected_locations,
    ["Actual price"],
)

# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure6_EN.html", "w", encoding="utf-8") as f:
    f.write(html_str)

# Exporting as a PNG file
# fig.write_image("Charts/Figure6_EN.png")

# Previewing
fig

### Danish

In [60]:
fig = plot_price(
    sales_data,
    "M2AffordedTotal",
    "dk",
    selected_locations,
    ["Actual price"],
)

# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure6_DK.html", "w", encoding="utf-8") as f:
    f.write(html_str)

# Exporting as a PNG file
# fig.write_image("Charts/Figure6_DK.png")

# Previewing
fig

## Figure 7

In [61]:
# Creating the chart
selected_locations = ["København", "Frederiksberg", "Aarhus", "Odense", "Aalborg"]
# chart_title_en = "Average price per m² for Denmark's four largest cities over time"
# chart_title_dk = "Gennemsnitlig pris pr. m² over tid i Danmarks fire største byer"

### English

In [62]:
fig = plot_price(
    sales_data,
    "AvgSalesPrice",
    "en",
    selected_locations,
    ["Actual price"],
)

# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure7_EN.html", "w", encoding="utf-8") as f:
    f.write(html_str)

# Exporting as a PNG file
# fig.write_image("Charts/Figure7_EN.png")

fig

### Danish

In [63]:
fig = plot_price(
    sales_data,
    "AvgSalesPrice",
    "dk",
    selected_locations,
    ["Actual price"],
)

# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure7_DK.html", "w", encoding="utf-8") as f:
    f.write(html_str)

# Exporting as a PNG file
# fig.write_image("Charts/Figure7_DK.png")

fig

## Figure 8

In [64]:
# Creating the chart
selected_locations = ["København", "Frederiksberg", "Aarhus", "Odense", "Aalborg"]
# chart_title_en = "Average price per m² for Denmark's four largest cities over time (incl. predictions)"
# chart_title_dk = "Gennemsnitlig pris pr. m² over tid i Danmarks fire største byer (inkl. prognosetal)"

### English

In [65]:
fig = plot_price(
    sales_data,
    "AvgSalesPrice",
    "en",
    selected_locations,
    ["Actual price", "Predicted price"],
)

# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure8_EN.html", "w", encoding="utf-8") as f:
    f.write(html_str)

# Exporting as a PNG file
# fig.write_image("Charts/Figure8_EN.png")

# Previewing
fig

### Danish

In [66]:
fig = plot_price(
    sales_data,
    "AvgSalesPrice",
    "dk",
    selected_locations,
    ["Actual price", "Predicted price"],
)

# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure8_DK.html", "w", encoding="utf-8") as f:
    f.write(html_str)

# Exporting as a PNG file
# fig.write_image("Charts/Figure8_DK.png")

# Previewing
fig

In [67]:
print("DONE.")

DONE.
